In [1]:
from PIL import Image
import numpy as np
import math
import random
import pandas as pd
import webcolors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

FOLDER_PATH = 'data/'

In [2]:
def load_image(filename: str, resize: tuple):
  img = Image.open(filename)
  img = img.resize(resize)
  return np.array(img).astype(np.float64)

In [3]:
def show_image(filename: str):
  img = Image.open(filename)
  img.show()

In [4]:
def euclidean_distance(p1, p2):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p1, p2)))

In [5]:
def manhattan_distance(p1, p2):
  return sum(abs(px - qx) for px, qx in zip(p1, p2))

In [6]:
def nearest_centroid(point, centroids, distance):
  flag = False
  idx_centroid, min_distance = 0, 0

  for i in range(len(centroids)):
    current_distance = distance(point, centroids[i])
    if not flag or current_distance <  min_distance:
      flag = True
      
      idx_centroid = i
      min_distance = current_distance

  return idx_centroid, min_distance

In [7]:
def lloyd(data, k, iters, type, distance):
  distance_func = euclidean_distance if distance == "euclidian" else manhattan_distance

  go = True
  epoch_c = 0
  error = 0

  cluster_array = np.zeros(len(data))

  centroids = random.choices(data, k=k)

  while go:
    for element, i in zip(data, range(len(cluster_array))):
      idx_centroid, min_distace = nearest_centroid(
        element, centroids, distance_func)

      error += min_distace
      cluster_array[i] = idx_centroid

    for i in range(k):
      cluster_average = []
      for element, j in zip(data, range(len(cluster_array))):
        if cluster_array[j] == i:
          cluster_average.append(element)

      centroids[i] = np.average(cluster_average, axis=0)
      
    epoch_c += 1
    go = False if epoch_c > iters else True

    return centroids, error



In [8]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

In [9]:
def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

In [10]:
img = "img_01.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'darkorange')
(None, 'maroon')
(None, 'chocolate')
(None, 'chocolate')
(None, 'maroon')


In [15]:
color_palletes

[array([164.72462203, 160.53887689, 160.74946004]),
 array([ 94.71673525, 108.11248285, 108.42729767]),
 array([25.46522487, 30.73900221, 33.45686901]),
 array([ 3.21895954, 18.39653179, 28.16      ]),
 array([65.57670353, 67.83660364, 66.60577952])]

In [11]:
img = "img_02.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'rosybrown')
(None, 'darkslategray')
(None, 'purple')
(None, 'indigo')
(None, 'slateblue')


In [ ]:
color_palletes

In [12]:
img = "img_03.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'darkgray')
(None, 'darkslategray')
(None, 'gray')
(None, 'darkolivegreen')
(None, 'rosybrown')


In [ ]:
color_palletes


In [13]:
img = "img_04.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'saddlebrown')
(None, 'dimgray')
(None, 'dimgray')
(None, 'dimgray')
(None, 'lightslategray')


In [ ]:
color_palletes


In [14]:
img = "img_05.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'darkgray')
(None, 'dimgray')
(None, 'black')
(None, 'black')
(None, 'darkslategray')


In [ ]:
color_palletes
